In [1]:
# Only Close
print("=" * 80)
print("{:=^80}" .format(" Only Use Close Data "))
print("=" * 80, end="\n\n")

# Seed
from numpy.random import seed
seed(1234)
from tensorflow import set_random_seed
set_random_seed(1234)

# Warning Message
import warnings
warnings.filterwarnings(action='ignore') 

# Data Loading
import pandas as pd
data = pd.read_csv("./data/df_final.csv")

# 필요한 칼럼 추출
data = data[["date", "open", "high", "low", "close", "code"]]

# Code 앞에 0 추가
data["code"] = data["code"].apply(lambda x : "{:0>6}" .format(x))

# Code type을 str로 변경
data["code"] = data["code"].apply(lambda x : str(x))

# 각 종목별 obs 개수
print("="*80)
print("{:=^74}" .format(" 각 종목별 obs 개수 "))
print("="*80)
print(data["code"].value_counts())

print("\n\n")

# 3년간의 데이터가 완전한 종목 코드
print("="*80)
print("{:=^67}" .format(" 3년간 데이터가 완전한 종목 코드 733obs "))
print("="*80)
FULL_DATA_CODE = data["code"].value_counts().index[0:2323]
print(FULL_DATA_CODE)

print("\n\n")

# 3년간의 데이터가 완전한 종목만 추출한 데이터
print("="*80)
print("{:=^61}" .format(" 3년간의 데이터가 완전한 종목만 추출한 데이터 "))
print("="*80)
data = data[data["code"].isin(FULL_DATA_CODE)]
print(data)

print("\n\n")

# RESULT DATASET 생성
FINAL_DATA = pd.DataFrame()

# 각 종목 별 Shift 생성
# 5일 전 종가로 5일 후 종가를 예측
temp_data = data                                                                                      
for i in FULL_DATA_CODE :                                                                      
    data = data[data["code"] == i]                                                                   
    for s in range(1,6):                                                                             
            data['{}{}'.format("close",s)] = data["close"].shift(s)                                        
    data = data.dropna().drop(["open", "high", "low", "close1", "close2", "close3", "close4", "date", "code"], axis=1)
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    scaler.fit(data)
    data = scaler.transform(data)
    data = pd.DataFrame(data, columns = ["close", "close5"])
    FINAL_DATA = FINAL_DATA.append(data)                                                             
    data = temp_data                                                                               
print("="*80)
print("{:=^66}" .format(" SHIFT 생성 : 5일 전 종가로 5일 후 종가를 예측 "))
print("="*80)
FINAL_DATA = FINAL_DATA[["close", "close5"]]
print("DATA Length : {}obs" .format(len(FINAL_DATA)))
print(FINAL_DATA[0:10])

# Data 새로 만들기
FINAL_DATA.to_csv("./data/ALLCODE_DATA.csv")

# 새로 만든 데이터 불러오기
data = pd.read_csv("./data/ALLCODE_DATA.csv")

# Feature 추출
data = data[['close', 'close5']]

print("\n\n")

# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[["close5"]], data[["close"]], test_size=0.10, random_state=42)

# Model Fitting
from sklearn.linear_model import LinearRegression
reg = LinearRegression(n_jobs = -1).fit(X_train, y_train)

print("="*80)
print("{:=^80}" .format(" Linear Regression "))
print("="*80)
print("Train Score : {}" .format(reg.score(X_train, y_train)))
print("Test Score : {}" .format(reg.score(X_test, y_test)))

# RMSE
import numpy as np
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_train, reg.predict(X_train))
rmse = mse ** 0.5
print("Train RMSE : {}" .format(rmse))
mse = mean_squared_error(y_test, reg.predict(X_test))
rmse = mse ** 0.5
print("Test RMSE : {}" .format(rmse))

##############################################################################
##############################################################################
##############################################################################
##############################################################################

# Sk 하이닉스 데이터 예측해보기

# Data Loading
data = pd.read_csv("./data/df_final.csv")

# 필요한 칼럼 추출
data = data[["date", "open", "high", "low", "close", "code"]]

# Code 앞에 0 추가
data["code"] = data["code"].apply(lambda x : "{:0>6}" .format(x))

# Code type을 str로 변경
data["code"] = data["code"].apply(lambda x : str(x))

print("\n\n\n\n")

# 3년간의 데이터가 완전한 종목만 추출한 데이터
print("="*80)
print("{:=^76}" .format(" SK 하이닉스 "))
print("="*80)
data = data[data["code"] == "000660"]
print(data)

print("\n\n")

# 각 종목 별 Shift 생성
# 5일 전 종가로 5일 후 종가를 예측
for s in range(1,6):                                                                             
        data['{}{}'.format("close",s)] = data["close"].shift(s)                                        
data = data.dropna().drop(["open", "high", "low", "close1", "close2", "close3", "close4", "date", "code"], axis=1)
print("="*80)
print("{:=^76}" .format(" SK 하이닉스 SHIFT DATA "))
print("="*80)
print("DATA Length : {}obs" .format(len(data)))
print(data[0:10])

print("\n\n")

FINAL_DATA = pd.DataFrame()
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(data)
data = scaler.transform(data)
data = pd.DataFrame(data, columns = ["close", "close5"])
FINAL_DATA = FINAL_DATA.append(data)
print("="*80)
print("{:=^76}" .format(" SK 하이닉스  MinMaxScaler "))
print("="*80)
FINAL_DATA = FINAL_DATA[["close", "close5"]]
print("DATA Length : {}obs" .format(len(FINAL_DATA)))
print(FINAL_DATA[0:10])

print("\n\n")

# 하이닉스 X, y
X = FINAL_DATA[["close5"]].values
y = FINAL_DATA[["close"]].values

# Sk 하이닉스 Score
print("="*80)
print("{:=^76}" .format(" SK하이닉스 Score "))
print("="*80)
print("Test Score : {}" .format(reg.score(X, y)))

# RMSE
import numpy as np
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y, reg.predict(X))
rmse = mse ** 0.5
print("Test RMSE : {}" .format(rmse))

============================= Only Use Close Data ==============================

============================== 각 종목별 obs 개수 ==============================
017940    733
023450    733
001380    733
025000    733
032350    733
         ... 
287310    494
570024    494
287300    494
287330    494
269620    493
Name: code, Length: 2539, dtype: int64



==================== 3년간 데이터가 완전한 종목 코드 733obs ====================
Index(['017940', '023450', '001380', '025000', '032350', '001465', '066980',
       '189300', '240810', '058470',
       ...
       '013990', '079810', '153360', '018120', '099410', '550011', '253150',
       '226340', '142280', '018500'],
      dtype='object', length=2323)



================= 3년간의 데이터가 완전한 종목만 추출한 데이터 =================
               date   open   high    low  close    code
0        2017-01-02  79905  81585  79170  79800  012320
1        2017-01-03  80535  80535  79170  79800  012320
2        2017-01-04  80535  80955  76965  77175  012320
3        2017-0

In [2]:
# With OHLC (open, high, low ,close)
print("=" * 80)
print("{:=^80}" .format(" With OHLC (open, high, low ,close) "))
print("=" * 80, end="\n\n")

# Seed
from numpy.random import seed
seed(1234)
from tensorflow import set_random_seed
set_random_seed(1234)

# Warning Message
import warnings
warnings.filterwarnings(action='ignore') 

# Data Loading
import pandas as pd
data = pd.read_csv("./data/df_final.csv")

# 필요한 칼럼 추출
data = data[["date", "open", "high", "low", "close", "code"]]

# Code 앞에 0 추가
data["code"] = data["code"].apply(lambda x : "{:0>6}" .format(x))

# Code type을 str로 변경
data["code"] = data["code"].apply(lambda x : str(x))

# 각 종목별 obs 개수
print("="*80)
print("{:=^74}" .format(" 각 종목별 obs 개수 "))
print("="*80)
print(data["code"].value_counts())

print("\n\n")

# 3년간의 데이터가 완전한 종목 코드
print("="*80)
print("{:=^67}" .format(" 3년간 데이터가 완전한 종목 코드 733obs "))
print("="*80)
FULL_DATA_CODE = data["code"].value_counts().index[0:2323]
print(FULL_DATA_CODE)

print("\n\n")

# 3년간의 데이터가 완전한 종목만 추출한 데이터
print("="*80)
print("{:=^61}" .format(" 3년간의 데이터가 완전한 종목만 추출한 데이터 "))
print("="*80)
data = data[data["code"].isin(FULL_DATA_CODE)]
print(data)

print("\n\n")

# RESULT DATASET 생성
FINAL_DATA = pd.DataFrame()

# 각 종목 별 Shift 생성
# 5일 전 종가로 5일 후 종가를 예측
temp_data = data                                                                                      
for i in FULL_DATA_CODE :                                                                      
    data = data[data["code"] == i]
    for k in ["open", "high", "low", "close"] : 
        for s in range(1,6):                                                                             
                data['{}{}'.format(k,s)] = data[k].shift(s)                                        
    data = data.dropna().drop(["open", "high", "low", "open1", "open2", "open3", "open4",\
                                "high1", "high2", "high3", "high4",\
                                "low1", "low2", "low3", "low4",\
                                "close1", "close2", "close3", "close4","date", "code"], axis=1)
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    scaler.fit(data)
    data = scaler.transform(data)
    data = pd.DataFrame(data, columns = ["close", "open5", "high5", "low5", "close5"])
    FINAL_DATA = FINAL_DATA.append(data)                                                             
    data = temp_data                                                                               
print("="*80)
print("{:=^66}" .format(" SHIFT 생성 : 5일 전 종가로 5일 후 종가를 예측 "))
print("="*80)
FINAL_DATA = FINAL_DATA[["close", "open5", "high5", "low5", "close5"]]
print("DATA Length : {}obs" .format(len(FINAL_DATA)))
print(FINAL_DATA[0:10])

# Data 새로 만들기
FINAL_DATA.to_csv("./data/ALLCODE_DATA.csv")

# 새로 만든 데이터 불러오기
data = pd.read_csv("./data/ALLCODE_DATA.csv")

# Feature 추출
data = data[["close", "open5", "high5", "low5", "close5"]]

print("\n\n")

# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[["open5", "high5", "low5", "close5"]],\
                                                    data[["close"]], test_size=0.10, random_state=42)

# Model Fitting
from sklearn.linear_model import LinearRegression
reg = LinearRegression(n_jobs = -1).fit(X_train, y_train)

print("="*80)
print("{:=^80}" .format(" Linear Regression "))
print("="*80)
print("Train Score : {}" .format(reg.score(X_train, y_train)))
print("Test Score : {}" .format(reg.score(X_test, y_test)))

# RMSE
import numpy as np
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_train, reg.predict(X_train))
rmse = mse ** 0.5
print("Train RMSE : {}" .format(rmse))
mse = mean_squared_error(y_test, reg.predict(X_test))
rmse = mse ** 0.5
print("Test RMSE : {}" .format(rmse))

##############################################################################
##############################################################################
##############################################################################
##############################################################################

# Sk 하이닉스 데이터 예측해보기

# Data Loading
data = pd.read_csv("./data/df_final.csv")

# 필요한 칼럼 추출
data = data[["date", "open", "high", "low", "close", "code"]]

# Code 앞에 0 추가
data["code"] = data["code"].apply(lambda x : "{:0>6}" .format(x))

# Code type을 str로 변경
data["code"] = data["code"].apply(lambda x : str(x))

print("\n\n\n\n")

# 3년간의 데이터가 완전한 종목만 추출한 데이터
print("="*80)
print("{:=^76}" .format(" SK 하이닉스 "))
print("="*80)
data = data[data["code"] == "000660"]
print("Data obs : {}" .format(len(data)))
print(data[0:10])

print("\n\n")

# 각 종목 별 Shift 생성
# 5일 전 종가로 5일 후 종가를 예측
FINAL_DATA = pd.DataFrame()
for k in ["open", "high", "low", "close"] : 
    for s in range(1,6):                                                                             
        data['{}{}'.format(k,s)] = data[k].shift(s)                                        
data = data.dropna().drop(["open", "high", "low", "open1", "open2", "open3", "open4",\
                           "high1", "high2", "high3", "high4",\
                           "low1", "low2", "low3", "low4",\
                           "close1", "close2", "close3", "close4", "date", "code"], axis=1)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(data)
data = scaler.transform(data)
data = pd.DataFrame(data, columns = ["close", "open5", "high5", "low5", "close5"])
FINAL_DATA = FINAL_DATA.append(data)                                                             
print("="*80)
print("{:=^76}" .format(" SK 하이닉스 SHIFT DATA "))
print("="*80)
print("DATA Length : {}obs" .format(len(data)))
print(data[0:10])

print("\n\n")

# 하이닉스 X, y
X = FINAL_DATA[["open5", "high5", "low5", "close5"]].values
y = FINAL_DATA[["close"]].values

# Sk 하이닉스 Score
print("="*80)
print("{:=^76}" .format(" SK하이닉스 Score "))
print("="*80)
print("Test Score : {}" .format(reg.score(X, y)))

# RMSE
import numpy as np
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y, reg.predict(X))
rmse = mse ** 0.5
print("Test RMSE : {}" .format(rmse))

====================== With OHLC (open, high, low ,close) ======================

============================== 각 종목별 obs 개수 ==============================
017940    733
023450    733
001380    733
025000    733
032350    733
         ... 
287310    494
570024    494
287300    494
287330    494
269620    493
Name: code, Length: 2539, dtype: int64



==================== 3년간 데이터가 완전한 종목 코드 733obs ====================
Index(['017940', '023450', '001380', '025000', '032350', '001465', '066980',
       '189300', '240810', '058470',
       ...
       '013990', '079810', '153360', '018120', '099410', '550011', '253150',
       '226340', '142280', '018500'],
      dtype='object', length=2323)



================= 3년간의 데이터가 완전한 종목만 추출한 데이터 =================
               date   open   high    low  close    code
0        2017-01-02  79905  81585  79170  79800  012320
1        2017-01-03  80535  80535  79170  79800  012320
2        2017-01-04  80535  80955  76965  77175  012320
3        2017-0

In [10]:
# 경동인베스트 데이터 예측해보기

# Data Loading
data = pd.read_csv("./data/df_final.csv")

# 필요한 칼럼 추출
data = data[["date", "open", "high", "low", "close", "code"]]

# Code 앞에 0 추가
data["code"] = data["code"].apply(lambda x : "{:0>6}" .format(x))

# Code type을 str로 변경
data["code"] = data["code"].apply(lambda x : str(x))

# 3년간의 데이터가 완전한 종목만 추출한 데이터
print("="*80)
print("{:=^76}" .format(" 경동인베스트 "))
print("="*80)
data = data[data["code"] == "035760"]
print("Data obs : {}" .format(len(data)))
print(data[0:10])

print("\n\n")

# 각 종목 별 Shift 생성
# 5일 전 종가로 5일 후 종가를 예측
FINAL_DATA = pd.DataFrame()
for k in ["open", "high", "low", "close"] : 
    for s in range(1,6):                                                                             
        data['{}{}'.format(k,s)] = data[k].shift(s)                                        
data = data.dropna().drop(["open", "high", "low", "open1", "open2", "open3", "open4",\
                           "high1", "high2", "high3", "high4",\
                           "low1", "low2", "low3", "low4",\
                           "close1", "close2", "close3", "close4", "date", "code"], axis=1)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(data)
data = scaler.transform(data)
data = pd.DataFrame(data, columns = ["close", "open5", "high5", "low5", "close5"])
FINAL_DATA = FINAL_DATA.append(data)                                                             
print("="*80)
print("{:=^76}" .format(" 경동인베스트 SHIFT DATA "))
print("="*80)
print("DATA Length : {}obs" .format(len(data)))
print(data[0:10])

print("\n\n")

# 하이닉스 X, y
X = FINAL_DATA[["open5", "high5", "low5", "close5"]].values
y = FINAL_DATA[["close"]].values

# Sk 하이닉스 Score
print("="*80)
print("{:=^76}" .format(" 경동인베스트 Score "))
print("="*80)
print("Test Score : {}" .format(reg.score(X, y)))

# RMSE
import numpy as np
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y, reg.predict(X))
rmse = mse ** 0.5
print("Test RMSE : {}" .format(rmse))

================================== 경동인베스트 ==================================
Data obs : 733
              date    open    high     low   close    code
310059  2017-01-02  162900  163100  160100  160100  035760
310060  2017-01-03  161800  161800  159300  160200  035760
310061  2017-01-04  160200  162100  158800  159000  035760
310062  2017-01-05  159000  160500  157000  157500  035760
310063  2017-01-06  157600  159600  157400  159000  035760
310064  2017-01-09  160000  160700  157700  159000  035760
310065  2017-01-10  159000  159000  156400  156500  035760
310066  2017-01-11  156600  158000  155900  157800  035760
310067  2017-01-12  157700  160100  156200  160100  035760
310068  2017-01-13  159900  160400  158000  160100  035760



============================ 경동인베스트 SHIFT DATA =============================
DATA Length : 728obs
      close     open5     high5      low5    close5
0  0.076812  0.104727  0.091034  0.098519  0.084783
1  0.058696  0.096727  0.082069  0.092593  0.085507
2 